<a href="https://colab.research.google.com/github/nan-park/cp1_project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [2]:
# (체크) 데이터 불러오는 것도 나중에 함수화 하기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/binary_dataset.csv')
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


In [4]:
# 학습/테스트 데이터 뒤섞기
def df_shuffle(df):
  return df.sample(frac=1).reset_index(drop=True)

# 특성, 타겟 나누기
def divide_xy(df):
  target = 'y'
  features = list(df.columns)
  features.remove(target)

  X = df[features]
  y = df[target]
  return X, y

# 학습/테스트 데이터 분리하기
def train_test_divide(X, y, test_size=0.2):  # X: pandas dataframe, y: pandas series
  length = len(y)
  test_index = int(length * test_size)

  X_test = X[:test_index]
  y_test = y[:test_index]

  X_train = X[test_index:]  
  y_train = y[test_index:]

  return X_train, y_train, X_test, y_test

In [5]:
def train_test_split(df, shuffle=True, test_size=0.2):
  if shuffle:
    df = df_shuffle(df)
  
  X, y = divide_xy(df)
  return train_test_divide(X, y, test_size=test_size)

In [6]:
X_train, y_train, X_test, y_test = train_test_split(df)

In [7]:
# 미니배치 배정 -> 미니배치 개수를 기준으로 가중치 개수 정해야 함
# 미니배치를 통해 결국에 GPU의 병렬처리를 활용해야 함
# 가중치, 편향 값 랜덤 지정(Xavier 초기화 참고)
# 인공신경망 층 클래스 정의 필요한가?(가중치, 편향 값 존재) -> W, b




In [8]:
# 미니배치 설정. 이게 그대로 인공 신경망에 들어감
def set_mini_batch(X, y): # train, test 들어올 예정
  # 4개씩 미니배치 설정. 나머지는 버리기
  length = len(y)
  num = length // 4 # 미니배치 개수
  X_batch_list = []
  y_batch_list = []
  for i in range(num):
    i = i * 4
    # 비복원 추출. 데이터가 적기 때문에 겹치지 않는 게 나을 듯.
    X_batch_list.append(X[i:i+4]) # index: 0~4, 4~8, 8~12, ...
    y_batch_list.append(y[i:i+4])
  return X_batch_list, y_batch_list

In [9]:
# 층의 노드 개수(n), 미니배치 하나 개수(4개) -> 가중치 4xn

# Xavier 초기화(활성화함수가 시그모이드일 때 잘 동작), 편향 포함한 후에 분리하기
# 이전 층 노드 개수가 n, 현재 층 노드 개수 m일 때, 표준편차가 2/루트(n+m)인 정규분포로 초기화

def initialize_parameter(n, m):
  init = np.random.normal(0, 2/((n+m)**2), (n, m+1))
  W = init[:, :-1]
  b = init[:, -1]
  return W, b

initialize_parameter(4, 5)

(array([[-0.01845518, -0.00589035, -0.00232582, -0.02114563,  0.02967501],
        [-0.02723593,  0.00513361,  0.00450786,  0.01325056,  0.01408623],
        [ 0.01523808,  0.02884191,  0.00415821, -0.00677612, -0.04263435],
        [ 0.05732083, -0.02254164,  0.00265957,  0.05178278,  0.00387155]]),
 array([ 0.02080942,  0.00369074, -0.01386633,  0.01299552]))

In [10]:
class Layer():  # 은닉층, 출력층
  def __init__(self, node_num, activation='linear'):
    self.node_num = node_num
    self.activation = activation
    # self.W, self.b = initialize_parameter(self.pro_node_num, self.node_num)  # (체크)이전 층 정보를 어떻게 가져오지?
    # (체크) activation, input, W, b, next
    self.prev = None
    self.next = None

  def set_weights(self):
    if self.prev is not None:
      prev_node_num = self.prev.node_num
      self.W, self.b = initialize_parameter(prev_node_num, self.node_num)
  
  @property
  def X(self):
    return self.X
  @X.setter
  def X(self, value):
    self.X = value

class Dense(Layer):
  def __init__(self, node_num, activation='linear'):
    # super().__init__(self, node_num)
    self.node_num = node_num
    self.activation = activation
    self.prev = None
    self.next = None
  
  def output(self):
    # self.prev.output
    return np.dot(self.X, self.W) + self.b

class Input(Layer): # prev가 없음
  def __init__(self, node_num, activation='linear'):
    self.node_num = node_num
    self.activation = activation
    self.prev = None
    self.next = None


  # def output(self):
  #   return X

In [11]:
# Sequential([])에 layer 쌓고 서로 연결되도록 하기. 가중치 초기화 가능해야 함
class Sequential():
  def __init__(self, layer_list):
    if len(layer_list)==0:
      self.head = None
      self.tail = None
    elif len(layer_list)==1:
      self.head = layer_list[0]
      self.tail = layer_list[0]
    else:
      self.head = layer_list[0]
      iterator = self.head
      for layer in layer_list[1:-1]:
        layer.prev= iterator
        iterator.next = layer
        iterator = layer
      iterator.next = layer_list[-1]
      self.tail = layer_list[-1]
      self.tail.prev = iterator

    # 가중치, 편향 초기화
    iterator = self.head
    while iterator:
      iterator.set_weights()
      iterator = iterator.next

  @property
  def input(self):
    return self.input
  @input.setter
  def input(self, value):
    self.input = value
    self.input.X = value

  def output(self):
    
    

In [12]:
model = Sequential([Input(4), Dense(16), Dense(32), Dense(1, activation='sigmoid')])

In [13]:
# 미니배치 하나(4개) 데이터 넣어서 output 내기
